### PODŁĄCZENIE DO API

In [1]:
from __future__ import unicode_literals
import requests
from requests.auth import AuthBase
import datetime
from datetime import timedelta
import pyodbc

In [2]:
def set_day():
    today = datetime.date.today()
    weekend = set([5,6])
    yesterday = today - timedelta(days=1)
    if yesterday.weekday() not in weekend:
        yesterdays = today - timedelta(days=1)
        date = yesterdays.strftime('%Y-%m-%d')
    elif yesterday.weekday() == 5:
        yesterdays = today - timedelta(days=3)
        date = yesterdays.strftime('%Y-%m-%d')
    elif yesterday.weekday() == 6:
        yesterdays = today - timedelta(days=3)
        date = yesterdays.strftime('%Y-%m-%d')


    ereyesterday = today - timedelta(days=2)
    if ereyesterday.weekday() not in weekend:
        ereyesterdays = today - timedelta(days=2)
        ere = ereyesterdays.strftime('%Y-%m-%d')
    elif ereyesterday.weekday() == 5:
        ereyesterdays = today - timedelta(days=4)
        ere = ereyesterdays.strftime('%Y-%m-%d')
    elif ereyesterday.weekday() == 6:
        ereyesterdays = today - timedelta(days=4)
        ere = ereyesterdays.strftime('%Y-%m-%d')
    return date,ere,today

    twodayago = today - timedelta(days=3)
    twodays = twodayago.strftime('%Y-%m-%d')

    
date, ere, today = set_day()

In [3]:
print("date:", date, "ere:", ere, "today:", today )

date: 2021-04-23 ere: 2021-04-22 today: 2021-04-26


In [4]:
class OAuth2(requests.auth.AuthBase):
    def __init__(self, user, signature):
        self.signature=signature
        self.user=user
    def __call__(self, r: requests.Request):
        r.headers["x-signature"] = self.signature
        r.headers['Accept'] = 'application/json'
        r.headers['Content-Type'] = 'application/json'
        print(r.headers)
        return r

### PODŁĄCZENIE DO SQL

In [5]:
connection = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=xxx;DATABASE=xxx;UID=xxx;PWD=xxx')
cursor = connection.cursor()


### WYBRANIE DANYCH Z SQL DO JSON

In [6]:
def get_instruments():
    l = []
    cursor.execute("""SELECT distinct p.isin FROM TAM_STAN_PORTFELA p 
    left join Tam_Instrument i on i.Isin=p.Isin WHERE TYP = 'Tytuly uczestnictwa zagranicznego' 
    AND DATA_ANALIZY =? and p.Isin like 'IE%' and p.isin not in ('IE00B4613386') and 
    (i.Nazwa like '%BOND%' or i.Nazwa like '%CORP%' or i.Nazwa like '%GOVER%' or i.Nazwa like '%TIPS%' or i.Nazwa like '%Treasur%')""", ere)
    for row in cursor:
        instruments = row[0]
        l.append(instruments)
    return l

In [7]:
get_instruments()

['IE00B66F4759']

In [8]:
def create_json():
    column_names=("instrumentIdentifierName","instrumentIdentifierValue","exchangeIdentifierName","exchangeIdentifierValue","dataSource")
    json_response=[]
    for i in get_instruments():
         
            json_element={}
            json_element[column_names[0]]="isin"
            json_element[column_names[1]]=i
            json_element[column_names[2]]="MIC"
            json_element[column_names[3]]="XEUR"
            json_element[column_names[4]]="ISHARES"
            json_response.append(json_element)
    return json_response

In [9]:
 print(json.dumps(create_json(), indent=5))

[
     {
          "instrumentIdentifierName": "isin",
          "instrumentIdentifierValue": "IE00B66F4759",
          "exchangeIdentifierName": "MIC",
          "exchangeIdentifierValue": "XEUR",
          "dataSource": "ISHARES"
     }
]


### Pobranie danych z API

In [1]:
url = 'https://xxx/api/v1/storage/duration/last?apiKey=xxx'
try:
    response_dur = requests.post(url, auth=OAuth2('xxx/xxx==',
                                                  'xxx'), json={"requests": create_json()}, timeout=None)

except requests.exceptions.HTTPError as errh:
    print("Http Error:", errh)
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
except requests.exceptions.RequestException as err:
    print("OOps: Something Else", err)
except Exception:
    pass


print(response_dur.status_code)
print(response_dur.text)

NameError: name 'requests' is not defined

### WYBRANIE DANYCH Z JSONA

In [11]:
d = response_dur.json()

def duration(isin):
   
    l = 0
    for i in d['responses']:
        if isin == d['responses'][l]['instrument']['instrumentIdentifierValue']:
            dur = d['responses'][l]['duration'][0]['value']
            return dur
        elif isin != d['responses'][l]['instrument']['instrumentIdentifierValue']:
            l+=1
            

In [12]:
print(duration('IE00B66F4759'))

3.21


### INSERT SQL

In [13]:

insert_query ='''insert into tam_wskaznik_instrument (Id_wskaznika, Isin, Data,Wartosc,Okres,Data_od,Id_funduszu) 
VALUES (?,?,?,?,?,?,?);'''


def create_lista_insert_MD():
    lista_ishares = []
    cursor.execute("""SELECT distinct p.id_funduszu, p.isin FROM TAM_STAN_PORTFELA p 
    left join Tam_Instrument i on i.Isin=p.Isin WHERE TYP = 'Tytuly uczestnictwa zagranicznego' 
    AND DATA_ANALIZY =? and p.isin not in ('IE00B4613386') and p.Isin like 'IE%' and 
    (i.Nazwa like '%BOND%' or i.Nazwa like '%CORP%' or i.Nazwa like '%GOVER%' or i.Nazwa like '%TIPS%')""", ere)

    for row in cursor:
        instruments = [row[0], row[1]]
        lista_ishares.append(instruments)


    elements = []
    p = 0
    for i in lista_ishares:
        elements_ishares = []
        elements_ishares.append(13)
        elements_ishares.append(lista_ishares[p][1])
        elements_ishares.append(date)
        elements_ishares.append(duration(lista_ishares[p][1]))
        elements_ishares.append('D')
        elements_ishares.append(date)
        elements_ishares.append(str(lista_ishares[p][0]))
        elements.append(elements_ishares)
        p+=1
    return elements

def create_lista_insert_D():
    lista_ishares = []
    cursor.execute("""SELECT distinct p.id_funduszu, p.isin FROM TAM_STAN_PORTFELA p 
    left join Tam_Instrument i on i.Isin=p.Isin WHERE TYP = 'Tytuly uczestnictwa zagranicznego' 
    AND DATA_ANALIZY =? and p.isin not in ('IE00B4613386') and p.Isin like 'IE%' and 
    (i.Nazwa like '%BOND%' or i.Nazwa like '%CORP%' or i.Nazwa like '%GOVER%' or i.Nazwa like '%TIPS%')""", ere)

    for row in cursor:
        instruments = [row[0], row[1]]
        lista_ishares.append(instruments)


    elements = []
    p = 0
    for i in lista_ishares:
        elements_ishares = []
        elements_ishares.append(11)
        elements_ishares.append(lista_ishares[p][1])
        elements_ishares.append(date)
        elements_ishares.append(duration(lista_ishares[p][1]))
        elements_ishares.append('D')
        elements_ishares.append(date)
        elements_ishares.append(str(lista_ishares[p][0]))
        elements.append(elements_ishares)
        p+=1
    return elements
    

def create_lista_insert_fixed():
    insert_date_fixed = []
    cursor.execute("""SELECT distinct id_funduszu,isin FROM TAM_STAN_PORTFELA 
    where isin in ('LU1681041205','LU1686830909','IE00B4613386','FR0010892745','FR0011607084',
    'PLINPZUINOST','PLINPZUOBLPL','PLINPZURYNWS','LU1681041205','LU1686830909','IE00B4613386','FR0010892745','FR0011607084') and data_analizy = ?""", ere)
    for row in cursor:
        if row[1] == 'LU1681041205':
            instrument_MD = [13,row[1],date, 9.32,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 9.32,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'LU1686830909':
            instrument_MD = [13,row[1],date, 9.49,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 9.49,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'IE00B4613386':
            instrument_MD = [13,row[1],date, 6.45,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 6.45,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'FR0010892745':
            instrument_MD = [13,row[1],date, 7.45,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 7.45,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'FR0011607084':
            instrument_MD = [13,row[1],date, -15.28,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, -15.28,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'PLINPZUINOST':
            instrument_MD = [13,row[1],date, 0.25,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 0.25,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'PLINPZUOBLPL':
            instrument_MD = [13,row[1],date, 4.35,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 4.35,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)
        elif row[1] == 'PLINPZURYNWS':
            instrument_MD = [13,row[1],date, 8.77,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_MD)
            instrument_D = [11,row[1],date, 8.77,'D',date, int(row[0])]
            insert_date_fixed.append(instrument_D)

    return insert_date_fixed
            

insert_date_changed_MD =create_lista_insert_MD() 
insert_date_changed_D =create_lista_insert_D()

for row in insert_date_changed_MD:
    values = (row[0],row[1],row[2],row[3],row[4],row[5],row[6])
    cursor.execute(insert_query,values)

for row in insert_date_changed_D:
    values = (row[0],row[1],row[2],row[3],row[4],row[5],row[6])
    cursor.execute(insert_query,values)

connection.commit()



In [14]:
create_lista_insert_MD()

[[13, 'IE00B66F4759', '2021-04-23', 3.21, 'D', '2021-04-23', '114'],
 [13, 'IE00B66F4759', '2021-04-23', 3.21, 'D', '2021-04-23', '134']]